Recontrucción de nube de test

In [21]:
import os
import numpy as np

#dataset_folder = 'split_de_paris10clases/reduced25k/'
#save_dir = 'split_de_paris10clases/reduced25k/normalized/'


dataset_folder = r'test_split_normalized/normalized_withtransform' 
save_dir = r'test_split_normalized/normalized_withtransform/reconstruida' 

datos_fragmentos = []

#savedir = 'test_split_normalized/'  #'procesadas_main/'

for filename in os.listdir(dataset_folder):

    f = os.path.join(dataset_folder, filename)


    if os.path.isfile(f):
        #print(f)
        
        data = np.load(f, allow_pickle=True)

        x = data[0]
        y = data[1]
        z = data[2]
        r = data[3]
        c = data[4]
        transformada = data[5]
        """
        max_x_norm = data[:, 5]
        max_y_norm = data[:, 6]
        min_x = data[:, 7]
        min_y = data[:, 8]
        min_z = data[:, 9]
        """

        #l = data[:, 5]


        max_x_norm = transformada[0]
        max_y_norm = transformada[1]
        min_x = transformada[2]
        min_y = transformada[3]
        min_z = transformada[4]


        print(max_x_norm)
        #print("-------------")
        #print(print(x))
        #print("-------------")
        x_original = x * max_x_norm
        x_original = x_original + min_x
        #print(print(x_original))

        y_original = y * max_y_norm
        y_original = y_original + min_y

        z_original = z * 100
        z_original = z_original + min_z
        

        """
        range_to_normalize = (0,1)
        x_normalized = (x-min(x))
        x_normalized = x_normalized/max(x_normalized)

        y_normalized = (y-min(y))
        y_normalized = y_normalized/max(y_normalized)

        z_normalized = (z-min(z))
        z_normalized = z_normalized/100
        """

        #guardar -> max(x_normalized), max(y_normalized), min(x), min(), min(z)

        #transformada = np.array([max(x_normalized), max(y_normalized), min(x), min(y), min(z)]).T


        #x = x_normalized * max(x_normalized) + min(x)
        #y = y_normalized * max(y_normalized) + min(y)
        #z = z_normalized * 100 + min(z)

        
        #AQUI GUARDAS LOS VALORES NEDESARIOS 1 ficheros por cada nube. Despues recorres igual y puedes invertir... puede ser interesante.
        #completamente hacer inversa, sunar y multiplicar y deberia tirar...


        #x_normalized = #normalize(x, range_to_normalize[0], range_to_normalize[1])
        #y_normalized = #normalize(y, range_to_normalize[0], range_to_normalize[1])
        #z_normalized = #normalize(z, range_to_normalize[0], range_to_normalize[1])

        #xyzrc_normalized = np.array([x_normalized,y_normalized,z_normalized,r,c,max(x_normalized), max(y_normalized), min(x), min(y), min(z)]).T

        xyzrc_normalized = np.array([x_original,y_original,z_original,r,c]).T
        #print(xyzrc_normalized)

            
        datos_fragmentos.append(xyzrc_normalized)

# concatenamos todos los datos de los fragmentos en una única matriz
nube_de_puntos = np.vstack(datos_fragmentos)
print(nube_de_puntos[0])
#print(max(y)-min(y))
np.save(os.path.join(save_dir, "nube_recontruida"), nube_de_puntos)



53.070000038831495
40.710000076913275
49.72000000497792
53.400000056193676
53.41999984352151
53.41999984352151
50.95000011718366
53.42000010475749
53.42000010475749
53.42000010475749
53.42000010475749
53.42000010475749
53.42000010475749
53.410000080475584
53.410000080475584
53.41999984352151
53.41999984352151
53.41999984352151
53.41999984352151
53.41999984352151
53.41999984352151
53.41999984352151
53.41999984352151
53.41999984352151
51.06000012304867
53.42000010475749
53.42000010475749
53.42000010475749
53.42000010475749
53.42000010475749
53.410000080475584
53.42000010475749
53.42000010475749
53.410000080475584
53.42000010475749
53.400000056193676
53.42000010475749
53.4099998192396
53.4099998192396
53.4099998192396
53.4099998192396
53.4099998192396
53.4099998192396
53.4099998192396
53.4099998192396
53.4099998192396
53.4099998192396
53.379999746452086
53.390000031969976
53.4099998192396
53.4099998192396
53.410000080475584
53.42000010475749
53.42000010475749
53.42000010475749
53.42000010

Mostrador de ficheros .npy

In [22]:
import os
import numpy as np
import laspy
import math
import open3d as o3d

MisColores = {
    1: np.array([0.5,0.25,0]), #terreno
    2: np.array([0.1,0.6,0]),  #vegetación
    3: np.array([0.99,0,0]), #coche
    4: np.array([0.5,0.5,0.5]), #torre
    5: np.array([0.99,0,0.99]), #cable
    6: np.array([0.6,0.4,0.6]), #valla/muro
    7: np.array([0.99,0.99,0]), #farola
    8: np.array([0.15,0.09,0.9]), #edificio
}

MisColores = {
    0: np.array([0.5,0.25,0]), #terreno
    1: np.array([0.1,0.6,0]),  #vegetación
    2: np.array([0.99,0,0]), #coche
    3: np.array([0.5,0.5,0.5]), #torre
    4: np.array([0.99,0,0.99]), #cable
    5: np.array([0.6,0.4,0.6]), #valla/muro
    6: np.array([0.99,0.99,0]), #farola
    7: np.array([0.15,0.09,0.9]), #edificio
}


data = np.load('test_split_normalized/normalized_withtransform/reconstruida/nube_recontruida.npy')

#las_dir = 'procesadas_main/'
#for filename in os.listdir(las_dir):
    #if filename.endswith('.npy'):
#data = np.load(las_dir+filename)   #'procesadas_main/d2RCD-4-grafos-joint_025.npy')


#data = np.load('procesadas_main/d2RCD-4-grafos-joint_NO16384_001.npy')   
#data = np.load('procesadas_main/d2RCD-4-grafos-joint_NO16384_001.npy') 
#data = np.load('procesadas_main/buenaschano/1-celra-grafos-joint-3.npy')
x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
c = data[:, 4].astype(int)
r = data[:, 3]

points = data[:, 0:3]
print(x.shape)
print(np.unique(c))
# Assign colors to each point based on the corresponding value in the c array
colors = np.zeros_like(points)
for i, value in MisColores.items():
    mask = (c == i)
    colors[mask] = value

cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)
cloud.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([cloud])

(3571768,)
[0 1 2 3 4 5 6 7]
